<a href="https://colab.research.google.com/github/SubramanyaJ/23CS5PCAIN/blob/main/L1_Genetic_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd
import random

def sel_indiv(df):
  return [row['Popul'] for _, row in df.iterrows() for _ in range(int(row['ActualCnt']))]

def cross(mating_pool):
  random.shuffle(mating_pool)
  if len(mating_pool) % 2 != 0: mating_pool.append(random.choice(mating_pool))
  offspring, mating_pool_data, crossover_data = [], [], []
  for i in range(0, len(mating_pool), 2):
    p1, p2 = int(mating_pool[i]), int(mating_pool[i+1])
    bp1, bp2 = bin(p1)[2:].zfill(8), bin(p2)[2:].zfill(8)
    cp = random.randint(1, len(bp1) - 1)
    o1_binary, o2_binary = bp1[:cp] + bp2[cp:], bp2[:cp] + bp1[cp:]
    o1_decimal, o2_decimal = int(o1_binary, 2), int(o2_binary, 2)
    offspring.extend([o1_decimal, o2_decimal])
    mating_pool_data.extend([{'Subject No': i + 1, 'Value': p1, 'Mating Pool (Binary)': bp1, 'Crossover Point': cp, 'Fitness': p1**2},
                             {'Subject No': i + 2, 'Value': p2, 'Mating Pool (Binary)': bp2, 'Crossover Point': cp, 'Fitness': p2**2}])
    crossover_data.append({'Parent 1': p1, 'Parent 2': p2, 'Binary Parent 1': bp1, 'Binary Parent 2': bp2, 'Crossover Point': cp,
                           'Offspring 1 Binary': o1_binary, 'Offspring 2 Binary': o2_binary, 'Offspring 1 Decimal': o1_decimal, 'Offspring 2 Decimal': o2_decimal})
  return offspring, pd.DataFrame(mating_pool_data), pd.DataFrame(crossover_data)

def mutate_offs(offspring, mut_rate):
  mut_offsp, mutation_data = [], []
  for i, indiv in enumerate(offspring):
    original_binary = bin(indiv)[2:].zfill(8)
    mutated_binary, mutated_indiv, mutation_happened = original_binary, indiv, False
    if random.random() < mut_rate:
      bin_list = list(original_binary)
      if bin_list:
          mut_point = random.randint(0, len(bin_list) - 1)
          bin_list[mut_point] = '1' if bin_list[mut_point] == '0' else '0'
          mutated_binary = "".join(bin_list)
          mutated_indiv = int(mutated_binary, 2)
          mutation_happened = True
    mut_offsp.append(mutated_indiv)
    mutation_data.append({'Subject No': i + 1, 'Offspring Before Mutation (Binary)': original_binary,
                           'Mutation Chromosome (Binary)': mutated_binary if mutation_happened else original_binary,
                           'Offspring After Mutation (Binary)': bin(mutated_indiv)[2:].zfill(8),
                           'X Value (Decimal)': mutated_indiv, 'Fitness': mutated_indiv**2})
  return mut_offsp, pd.DataFrame(mutation_data)

def gen_gen(popul, mut_rate, initial_popn):
    df_pop = pd.DataFrame({'Subject No': range(1, len(popul) + 1), 'Popul': popul, 'Initial Popn (Binary)': [bin(p)[2:].zfill(8) for p in initial_popn]})
    fit = [ind ** 2 for ind in popul]
    cumul = sum(fit)
    prob = [f / cumul for f in fit]
    perc_prob = [p * 100 for p in prob]
    exp = [len(popul) * p for p in prob]
    actual = [round(e) for e in exp]
    df_pop['Fit'], df_pop['Prob'], df_pop['Percentage Prob'], df_pop['ExpectCnt'], df_pop['ActualCnt'] = fit, prob, perc_prob, exp, actual
    mating_pool = sel_indiv(df_pop)
    offspring, df_mating_pool, df_crossover = cross(mating_pool)
    new_gen, df_mutation = mutate_offs(offspring, mut_rate)
    return new_gen, df_pop, df_mating_pool, df_crossover, df_mutation

popn = [12, 25, 5, 19]
initial_popn = popn[:]
curr_popul = popn[:]
best_sol, best_fit, fit_hist = None, -float('inf'), []
num_gens, mut_rate = 3, 0.01

for gen in range(num_gens):
    curr_popul, df_gen, df_mating_pool, df_crossover, df_mutation = gen_gen(curr_popul, mut_rate, initial_popn)
    fit_vals = [ind ** 2 for ind in curr_popul]
    best_fit_curr = max(fit_vals)
    best_ind_idx = fit_vals.index(best_fit_curr)
    best_ind_curr = curr_popul[best_ind_idx]
    if best_fit_curr > best_fit: best_fit, best_sol = best_fit_curr, best_ind_curr
    fit_hist.append(best_fit_curr)
    print(f"Gen {gen + 1}: Best Fit = {best_fit_curr}, Best Indiv = {best_ind_curr}, Popul = {curr_popul}")
    print("Generation Data:"); display(df_gen)
    print("Mating Pool Data:"); display(df_mating_pool)
    print("Crossover Data:"); display(df_crossover)
    print("Mutation Data:"); display(df_mutation)

print("\nGenetic Algorithm finished.")
print("Overall best solution found:", best_sol)
print("Fitness of the overall best solution:", best_fit)

Gen 1: Best Fit = 625, Best Indiv = 25, Popul = [25, 25, 25, 19]
Generation Data:


,Subject No,Popul,Initial Popn (Binary),Fit,Prob,Percentage Prob,ExpectCnt,ActualCnt
0,1,12,00001100,144,0.124675,12.467532,0.498701,0
1,2,25,00011001,625,0.541126,54.112554,2.164502,2
2,3,5,00000101,25,0.021645,2.164502,0.086580,0
3,4,19,00010011,361,0.312554,31.255411,1.250216,1


Mating Pool Data:


,Subject No,Value,Mating Pool (Binary),Crossover Point,Fitness
0,1,25,00011001,2,625
1,2,25,00011001,2,625
2,3,19,00010011,3,361
3,4,25,00011001,3,625


Crossover Data:


...

Crossover Data:


,Parent 1,Parent 2,Binary Parent 1,Binary Parent 2,Crossover Point,Offspring 1 Binary,Offspring 2 Binary,Offspring 1 Decimal,Offspring 2 Decimal
0,25,25,00011001,00011001,5,00011001,00011001,25,25
1,25,19,00011001,00010011,7,00011001,00010011,25,19


Mutation Data:


,Subject No,Offspring Before Mutation (Binary),Mutation Chromosome (Binary),Offspring After Mutation (Binary),X Value (Decimal),Fitness
0,1,00011001,00011001,00011001,25,625
1,2,00011001,00011001,00011001,25,625
2,3,00011001,00011001,00011001,25,625
3,4,00010011,00010011,00010011,19,361



Genetic Algorithm finished.
Overall best solution found: 25
Fitness of the overall best solution: 625
